In [98]:

"""
Creates session JSON according to: 
https://www.notion.so/vr-at-mpi/Experiment-Config-5df777d32e6943cabe28a6022ac72435 
"""


'\nCreates session JSON according to: \nhttps://www.notion.so/vr-at-mpi/Experiment-Config-5df777d32e6943cabe28a6022ac72435 \n'

In [99]:
# Imports and definitions:
import os
import json
import pandas as pd
import numpy as np
import random
import itertools


def create_trialsSeqVals(trialSettings, values, nTrials,
                         random_state=np.random.randint(1e5)):
    """Creates a list of dicts that fits into the `trialSequenceValues` value 
       part of the VRstereofem task generator.  

    Args:
        trialSettings (list of strings): defining the relevant settings (must \
                                         start with "block_num")
        values (dict): maps all possible values to each setting
        nTrials (int): number of trials in whole experiment
        random_state (int, optional): For replicatbility you could set a \
                                      random state. Do not do this for real \
                                      experiment.\
                                      Defaults to np.random.randint(1e5).

    Returns:
        list: list of dicts (key: "values", value: "value for each setting")
    """

    # set up DF:
    trials_df = pd.DataFrame(columns=trialSettings)
    
    # we make a random vector with reponse orders:
    assert (nTrials % len(values["order"]) == 0), 'nTrials must be multiple of  \
                                                   the emotion orders to get a \
                                                   balanced design.'
    order_vec = values["order"] * int(nTrials / len(values["order"]))
    random.shuffle(order_vec)

    # check how many repetitions of each given combination we have:
    nTrialsPerBlock = int(nTrials / len(values['block_num']))
    nCombos = len(values['stereo']) * len(values['avatar']) * len(values['emo'])
    assert (nTrialsPerBlock % nCombos == 0), 'Cannot produce balanced blocks.'
    nReps = int(nTrialsPerBlock / nCombos)


    for bn in values["block_num"]:
        for st in values["stereo"]:
            for av in values["avatar"]:
                for emo in values["emo"]:
                        for rep in range(nReps):
                            trials_df = trials_df.append({
                                            'block_num': bn,
                                            'stereo': st, 
                                            'avatar': av, 
                                            'emo': emo, 
                                            'order': order_vec.pop()}, 
                                            ignore_index=True)

    # shuffle within blocks:
    trialSeqVals_df = trials_df.groupby(['block_num']).sample(frac=1, 
                                                              random_state=random_state)

    trialSeqVals = []
    for idx in trialSeqVals_df.index:
        trialSeqVals.append({'values': [trialSeqVals_df.loc[idx, col] for col in trialSettings]})
    
    return trialSeqVals

In [100]:

## Specify relevant values:

fpath_out = './configFiles/'
fname_out = "experimentConfig_72trials.json"

participant = dict()

participant = {
	'ID': "test01",
	'age': 32
}

experimenter = "AN"
sessionNumber = 1

nTrials = 72 # must be a multiple of 24
nBlocks = 3 # this has to be nTrials / 24

# This instruction will be shown before each block
blockInstructionDef = "Please rate which emotion was shown by the face. \n \
					 Please always keep your eyes on the central target."

# after which blocks do we want to have a break (probably after every)
breaksAfterBlock = [i for i in range(1, nBlocks+1)]

# Which emotions will be displayed:
emotionNames = "neutral,happy,angry,surprised"


# Define trial settings:

# column names for the trial settings (must start with "block_num"!):
trialSettings = ["block_num","stereo","avatar","emo","order"]

# values:
values = dict.fromkeys(trialSettings)
# automatically generate block numbers:
values["block_num"] = [str(i) for i in range(1, nBlocks+1)]
# 2 stereo vision conditions:
values["stereo"] = ["0", "1"]
# 3 different avatars:
values["avatar"] = ["1", "2", "3"]
# 4 emotions:
values["emo"] = ["1", "2", "3", "4"]
# 4! (=24) different possibilities to order the 4 emotion buttons:
values["order"] = [''.join(map(str,s)) for s in itertools.permutations(range(1,len(values['emo'])+1), 4)]


In [101]:

# From above values, define the dictionnary that we will serialze into the JSON:

ses_dict = dict()

ses_dict = {
	"experimenter": experimenter,
	"participantID": participant['ID'],
	"sessionNumber": sessionNumber,
	"participantInfo":[
		{"type":"string","key":"Age","value":participant['age']}
	],
	"sessionSettings":[
		{"type":"strings","key":"emotionNames","value": emotionNames},
	],
	"blockInstructions":[
		{"type":"string","key":"1","value": blockInstructionDef}] * nBlocks,
	"breakAfter": breaksAfterBlock,
	"trialSequenceKeys":["block_num","stereo","avatar","emo","order"],
	"trialSequenceValues": create_trialsSeqVals(trialSettings, values, nTrials)
}

72


In [102]:
#Write the info as JSON to disk:
fpath = os.path.join(fpath_out, fname_out)

if not os.path.exists(fpath_out):
    os.makedirs(fpath_out)

if not os.path.exists(fpath):
    print(f"Writing {fname_out} to {os.path.abspath(fpath_out)}.")
    with open(fpath, "w") as write_file:
        json.dump(ses_dict, write_file, indent=4)
else:
    print(f'A config file with name {fname_out} exists already in' \
        f'{os.path.abspath(fpath_out)}. If you want to overwrite, delete the ' \
        'old file first and run this script again.')

Writing experimentConfig_72trials.json to c:\Users\Felix\Seafile\Experiments\ThesisAmmara\Code\Python\Experiment\configFiles.
